# Practica 2
- **Alumna:** Enya Quetzalli Gómez Rodríguez *(Eduardo Gómez Rodríguez)*
- **Profesora:** Olga Kolesnikova
- **Escuela:** Escuela Superior de Cómputo del IPN
- **Grupo:** 3CV9
- **Semestre:** 2020/2

## Lematización con etiquetas y obtener similaridad
**Instrucciones:**
    Obtener la similaridad de nuestro texto pero esta ocasión con nuestro texto lematizado.

In [1]:
import nltk

In [2]:
def clean_html(txt):
	from bs4 import BeautifulSoup
	return BeautifulSoup(txt,'lxml').get_text().lower()

In [3]:
def split_text(txt):
	return txt.replace('/', ' ').replace('.', ' ').replace('-', ' ')

In [4]:
def delete_trash(txt):
	import re
	good = {'\n'}
	for i in "abcdefghijklmnopqrstuvwxyz áéíóúñü":
		good.add(i)
	ans = ""
	for c in txt:
		if c in good:
			ans += c
	return ans

In [5]:
def delete_stopwords(txt):
	from nltk.corpus import stopwords
	ans = []
	stp = stopwords.words()
	for w in txt:
		if w not in stp:
			ans.append(w)
	return ans

In [6]:
def get_vocabulary(tokens):
	vocabulary = sorted(set(tokens))
	return vocabulary

In [7]:
def get_contexts(wordList, windowSize=4):
    contexts = dict()
    index = 0
    for index in range(len(wordList)):
        word = wordList[index]
        if word not in contexts:
            contexts[word] = []
        start = max(0, index - windowSize)
        end = min(index + windowSize, len(wordList) - 1)
        for i in range(start, end + 1):
            if i != index:
                contexts[word].append(wordList[i])
    return contexts

In [8]:
def load_lemmas_raw():
    with open("./files/lemmas.txt", 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    
    lemmas = {}
    for line in lines:
        if line != "":
            words = [word.strip() for word in line.split()]
            wordform = words[0].replace("#", "")
            lemmas[wordform] = words[-1]
    return lemmas

In [9]:
def replace_with_lemmas(tokens):
    lemmas = loadFromJson("lemmas_dic.json")
    lemmatized_tokens = []
    for word in tokens:
        if word in lemmas.keys():
            lemmatized_tokens.append(lemmas[word])
        else:
            lemmatized_tokens.append(word)
    return lemmatized_tokens

In [10]:
def dumpToJson(obj, name):
    import json
    with open("./files/"+name, 'w', encoding="utf8") as outfile:
        json.dump(obj, outfile, indent=4)

In [11]:
def loadFromJson(name):
    import json
    with open("./files/"+name, 'r', encoding="utf8") as f:
        return json.load(f)

In [12]:
tokens = loadFromJson("generate_lemmatized_tokens.json")
vocabulary = get_vocabulary(tokens)
contexts = loadFromJson("contexts_generate_lemmas.json")

Ahora que hemos recalculado los contexttos de cada palabra con nuestros tokens lematizados, probaremos nuevamente la similaridad.

In [13]:
import numpy
vectors = dict()
for target, context in contexts.items():
    cF = dict()
    for word in context:
        if word not in cF:
            cF[word] = 0
        cF[word] += 1
    l = numpy.zeros(len(vocabulary))
    for i in range(len(vocabulary)):
        word = vocabulary[i]
        if word in cF:
            l[i] = cF[word]
    vectors[target] = l

In [14]:
ans = []
v1 = vectors["acero"]
for word in vocabulary:
    v2 = vectors[word]
    cosine = numpy.dot(v1, v2) / numpy.sqrt(numpy.sum(v1 ** 2)) / numpy.sqrt(numpy.sum(v2 ** 2))
    ans.append((word, cosine))
ans.sort(key = lambda x: x[1], reverse=True)

In [15]:
with open("./p2_docs/similarity.txt", "w", encoding="utf8") as f:
    for par in ans:
        f.write(F"{par[0]} {par[1]}\n")

In [16]:
for par in ans[:30]:
    print(F"{par[0]} {par[1]}\n")

acero 0.9999999999999999

pesado 0.7499999999999999

cobre 0.5590169943749473

trasladar 0.5303300858899106

cable 0.49999999999999994

microprocesadores 0.44721359549995787

tubo 0.44721359549995787

vacío 0.4166666666666667

desmaterializado 0.37499999999999994

fibra 0.24999999999999997

penetración 0.24999999999999997

sostenible 0.22360679774997894

sino 0.20887610455277092

desplome 0.2080125735844609

paralelamente 0.2041241452319315

cubrir 0.1599005372667078

depresión 0.15811388300841894

frijol 0.15811388300841894

consumo 0.1436739427831727

finalizar 0.1386750490563073

luz 0.1386750490563073

tan 0.1358036190637778

alrededor 0.13245323570650436

externo 0.13130643285972254

producción 0.12830005981991682

mitad 0.12803687993289595

abocar 0.12499999999999999

aburrimiento 0.12499999999999999

acosar 0.12499999999999999

adepto 0.12499999999999999



Como podemos observar, ahora la palabra **cobre** paso de tener *0.49* de similitud a *0.55*, lo cual fué sufienciente para pasar a ser la segunda palabra más similar a **acero**, por lo que podemos notar una mejora en nuestro programa de similitud gracias a la lematización. 